In [ ]:
import csv
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import config
from tensorboardX import SummaryWriter

In [ ]:
class MLPb(nn.Module):
    def __init__(self, config):
        super(MLP, self).__init__()
        self.config = config
        self.model = nn.Sequential(
            nn.Linear(784, config.num_neorons_l1),
            nn.ReLU(),
            nn.Linear(config.num_neorons_l1, config.num_neorons_l2),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.Linear(config.num_neorons_l2, 10)),
        
    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
class MLPa(nn.Module):
    def __init__(self, config):
        super(MLP, self).__init__()
        self.config = config
        self.model = nn.Sequential(
            nn.Linear(784, config.num_neorons_l1),
            nn.ReLU(),
            nn.Linear(config.num_neorons_l1, config.num_neorons_l2),
            nn.ReLU(),
            nn.Linear(config.num_neorons_l2, 10)) 
        
    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        # Logistic Regression
        self.clf = nn.Linear(128, 10)

    def forward(self, x):
        return self.clf(self.conv(x).squeeze())   

In [ ]:
def build_model(config):
    if config.model_type == MLPa():
        model = MLPa(config)
    elif config.model_type == MLPb():
        model = MLPb(config)
    elif config.model_type == CNN():
        model = CNN(config)
    model.apply(weights_init)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=config.lr0, weight_decay = config.weight_decay)
    return model, criterion, optimizer

In [ ]:
torch.norm(input, p=2) #change input

In [ ]:
# USES TENSORBOARDX http://tensorboard-pytorch.readthedocs.io/en/latest/tensorboard.html

def train_model(model, config, criterion, optimizer):
    losses = 0
    writer = SummaryWriter('isaacsultan/IFT6135/Assignments/Assignment2/logs')
    # record the performance for this epoch
    train_data, valid_data, test_data = utility.load_dataset("mnist.pkl", config)
    
    for epoch in enumerate(config.num_epochs):
        # iterate over batches
        # the shape of train_data[0] must be 500 x 100 x 784
        # the shape of train_data[1] must be 500 x 100
        for i in range(train_data[0].shape[0]):
            optimizer.zero_grad()
            x = Variable(torch.from_numpy(train_data[0][i]))
            y = Variable(torch.from_numpy(train_data[1][i]))
            optimizer.zero_grad()
            # compute loss
            loss = criterion(model(x), y)
            # compute gradients and update parameters
            loss.backward()
            # take one SGD step
            optimizer.step()
        # record the performance for this epoch
        writer.add_scalars('learning_curve', {'train_loss' = train_loss, 
                                             'test_loss' = test_loss}, epoch)
        # print the results for this epoch
        print("Epoch {0} \n Train Loss : {1:.3f} \Test Loss : {2:.3f}".format(epoch, train_loss, test_loss))
        

In [ ]:
def main():
    args = utility.parse_args()
    config_type = args['configtype']
    config_file = args['configfile']
    pdb.set_trace()
    config = config.Configuration(config_type, config_file)
    model, criterion, optimizer = build_model(config)
    train_model(model, config, criterion, optimizer)


In [ ]:
if __name__ == '__main__':
    main()